In [1]:
from pathlib import Path

Next steps:

 - load ROIs (or create one with shape of the image)
 - make adaptations to database:
   - ~use Path objects to enable continous integration~
   - ~rename deepflash2_dir to segmentation_tool_dir~
   - create sorted list of all preprocessing steps (i.e. preprocessing strategies)
 - adapt main
 - function that saves preprocessed images

 - Minimal preprocessing steps are:
    - save the "unprocessed" microscopy images to the preprocessed_dir
    - load the unprocessed ROIs into the database (create ROI with shape of image if whole image is to be analyzed)

In [13]:
hunde = list()
katzen = list()
for x in hunde:
    katzen.append(x)
    

In [14]:
katzen

[]

In [2]:
project_root_dir = Path('/mnt/c/Users/dsege/TEMP/test_project2/')

In [12]:
project_root_dir.joinpath(elem).as_posix()

'/mnt/c/Users/dsege/TEMP/test_project2/merge_error_example_file_id_0000_plane_5_centroid_x_4201_y_2426.png'

In [10]:
elem

PosixPath('/mnt/c/Users/dsege/TEMP/test_project2/merge_error_example_file_id_0000_plane_5_centroid_x_4201_y_2426.png')

In [10]:
project_root_dir.is_dir()

True

In [15]:
project_root_dir

PosixPath('/mnt/c/Users/dsege/TEMP/test_project2')

In [ ]:
self.project_root_dir + [elem for elem in subdirectories if 'microscopy' in elem][0] + '/'

In [27]:
subdirectories

['00_microscopy_images',
 '01_rois_to_analyze',
 '02_preprocessed_images',
 '03_deepflash2',
 '04_binary_segmentations',
 '05_instance_segmentations',
 '06_inspection',
 '07_results',
 'for_sofie',
 'for_sofie.zip',
 'intermediate',
 'merge_error_example_file_id_0000_plane_5_centroid_x_4201_y_2426.png']

In [13]:
import os
def listdir_nohidden(path):
    return [f for f in os.listdir(path) if f.startswith('.') == False]

In [16]:
subdirectories = listdir_nohidden(path = project_root_dir)

In [33]:
? microscopy_image_dir.mkdir

Signature:  microscopy_image_dir.mkdir(mode=511, parents=False, exist_ok=False)
Docstring: Create a new directory at this given path.
File:      ~/anaconda3/envs/DCLwidgets_clone/lib/python3.7/pathlib.py
Type:      method


In [28]:
subdirectory_attributes = {'rois_to_analyze_dir': {'foldername': '01_rois_to_analyze',
                                                   'key_substring': 'rois'},
                           'preprocessed_images_dir': {'foldername': '02_preprocessed_images',
                                                       'key_substring': 'preprocessed'},
                           'deepflash2_dir': {'foldername': '03_deepflash2',
                                              'key_substring': 'deepflash'},
                           'binary_segmentations_dir': {'foldername': '04_binary_segmentations',
                                                        'key_substring': 'binary'},
                           'instance_segmentations_dir': {'foldername': '05_instance_segmentations',
                                                          'key_substring': 'instance'},
                           'inspection_dir': {'foldername': '06_inspection',
                                              'key_substring': 'inspection'},
                           'results_dir': {'foldername': '07_results',
                                           'key_substring': 'results'}}


In [ ]:
for attribute_key in subdirectory_attributes.keys():
    elements_matching_key_substring = [elem for elem in subdirectories if subdirectory_attributes[attribute_key]['key_substring'] in elem]
    if len(elements_matching_key_substring) > 0:
        for matching_element in elements_matching_key_substring:
            if (project_root_dir.joinpath(matching_element).is_dir()) & (hasattr(self, attribute_key) == False):
                setattr(self, attribute_key, project_root_dir.joinpath(matching_element))
    if hasattr(self, attribute_key) == False:
        subdirectory_path = project_root_dir.joinpath(subdirectory_attributes[attribute_key]['foldername'])
        subdirectory_path.mkdir()
        setattr(self, attribute_key, subdirectory_path)
                
                

In [19]:
folder = [elem for elem in subdirectories if 'microscopy' in elem][0]

In [23]:
microscopy_image_dir = project_root_dir.joinpath('hund')

In [29]:
project_root_dir.joinpath('hund').is_dir()
    

False

In [26]:
microscopy_image_dir

PosixPath('/mnt/c/Users/dsege/TEMP/test_project2/hund')

In [ ]:
from findmycells.main import Project
import os
import pandas as pd
from datetime import datetime